In [1]:
#version de python
import sys
sys.version

'3.11.11 | packaged by conda-forge | (main, Dec  5 2024, 14:06:23) [MSC v.1942 64 bit (AMD64)]'

In [2]:
#version de pycaret
import pycaret
print(pycaret.__version__)

3.2.0


In [3]:
#version de mlflow
import mlflow
print(mlflow.__version__)

1.30.1


In [4]:
#changer de dossier
import os
os.chdir("C:/Users/lucie/venv")

In [5]:
import mlflow
import time
#importation de l'outil d'expérimentation de pycaret
from pycaret.classification import ClassificationExperiment
#mlflow.set_tracking_uri("file:///C:/Users/lucie/venv/mlruns")
mlflow.set_tracking_uri("http://127.0.0.1:5000")


In [6]:
#chargement des données 
import pandas as pd
data = pd.read_csv('Loan_Data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               10000 non-null  int64  
 1   credit_lines_outstanding  10000 non-null  int64  
 2   loan_amt_outstanding      10000 non-null  float64
 3   total_debt_outstanding    10000 non-null  float64
 4   income                    10000 non-null  float64
 5   years_employed            10000 non-null  int64  
 6   fico_score                10000 non-null  int64  
 7   default                   10000 non-null  int64  
dtypes: float64(3), int64(5)
memory usage: 625.1 KB


In [ ]:
import mlflow
from pycaret.classification import ClassificationExperiment
import pandas as pd

# S'assurer que toute session MLflow précédente est fermée
if mlflow.active_run():
    mlflow.end_run()

# Session secondaire avec suivi MLflow
with mlflow.start_run(run_name="optimisation_modele"):
    
    #  Initialisation et configuration de la session secondaire
    start_time = time.time()
    session_bis = ClassificationExperiment()
    session_bis.setup(data, normalize=True, target='default', train_size=0.7,
                      data_split_stratify=True, fold=5, session_id=1,
                      log_experiment=True, experiment_name="optimisation_modele")
    duration = time.time() - start_time
    mlflow.log_param("duration_setup", duration)

    # Comparaison des modèles
    model_list = ['lr', 'nb', 'dt', 'rf', 'svm', 'lda']
    model_durations = {}

    for model in model_list:
        start_time = time.time()
        mdl = session_bis.create_model(model)
        duration = time.time() - start_time
        model_durations[model] = duration
        mlflow.log_param(f"duration_{model}", duration)

    #  Sélection du meilleur modèle (hors boucle)
    best_model = session_bis.compare_models(sort='Accuracy', include=model_list, verbose=True)
    print("Durées d'entraînement par modèle :", model_durations)

    # Vérifier le type de modèle sélectionné
    model_name = str(best_model)
    print(f"Modèle sélectionné : {model_name}")

    # Définir une grille d'hyperparamètres adaptée
    if "LogisticRegression" in model_name:
        param_grid = {'C': [0.01, 0.1, 1, 10, 100]}  # Paramètres valides pour LogisticRegression
    elif "DecisionTree" in model_name:
       param_grid = {'min_samples_split': [2, 10, 20], 'max_depth': [5, 10, None]}
    elif "RandomForest" in model_name:
       param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [5, 10, None]}
    else:
      param_grid = {}  # Si le modèle sélectionné n'est pas connu, ne pas tuner.

    #  Lancer l'optimisation des hyperparamètres uniquement si une grille est définie
    if param_grid:
       start_time = time.time()
       tuned_mybest, essais = session_bis.tune_model(best_model, optimize="Accuracy",
                                                  choose_better=True,
                                                  custom_grid=param_grid,
                                                  search_algorithm='grid',
                                                  return_tuner=True)
       duration = time.time() - start_time
       mlflow.log_param("duration_tune_model", duration)
       print("Modèle optimisé :", tuned_mybest)
    else:
        print("Aucun tuning appliqué, modèle non pris en charge.")
        tuned_mybest = best_model
 
    #  Enregistrer les meilleurs paramètres trouvés
    best_params = tuned_mybest.get_params()
    for param, value in best_params.items():
        mlflow.log_param(f"best_{param}", value)

    #  Évaluation sur l'échantillon de test
    start_time = time.time()
    predictions = session_bis.predict_model(tuned_mybest)
    duration = time.time() - start_time
    mlflow.log_param("duration_predict_model", duration)
    print(predictions)

    # Finalisation et ré-entrainement du modèle sur l'ensemble des données
    start_time = time.time()
    modele_definitif = session_bis.finalize_model(tuned_mybest)
    duration = time.time() - start_time
    mlflow.log_param("duration_finalize_model", duration)
    print(modele_definitif)

    import os
    # Passer à MLflow 2.20.2
    os.system("conda activate theo1_env")
    #  Enregistrement du modèle "optimisation-model" dans MLflow
    import mlflow.sklearn
    if hasattr(best_model, "estimators_"):
         best_model = best_model.estimators_[0]  # Prendre le premier modèle du pipeline
         mlflow.sklearn.log_model(best_model, "recherche-model")
         # Ajouter au registre MLflow
         mlflow.register_model(f"runs:/{mlflow.active_run().info.run_id}/optimisation-model", "optimisation-model")
        
         # Revenir à MLflow 1.30.1
         os.system("conda activate theo_env")
   
# Fin explicite de la session (facultatif ici car le 'with' gère la fermeture)
mlflow.end_run()


,Description,Value
0,Session id,1
1,Target,default
2,Target type,Binary
3,Original data shape,"(10000, 8)"
4,Transformed data shape,"(10000, 8)"
5,Transformed train set shape,"(7000, 8)"
6,Transformed test set shape,"(3000, 8)"
7,Numeric features,7
8,Preprocess,True
9,Imputation type,simple


2025/02/25 08:52:39 INFO mlflow.tracking.fluent: Experiment with name 'optimisation_modele' does not exist. Creating a new experiment.


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9986,1.0000,0.9923,1.0000,0.9961,0.9952,0.9953
1,0.9979,1.0000,0.9884,1.0000,0.9942,0.9929,0.9929
2,0.9950,1.0000,0.9730,1.0000,0.9863,0.9832,0.9834
3,0.9979,1.0000,0.9884,1.0000,0.9942,0.9929,0.9929
4,0.9986,1.0000,0.9923,1.0000,0.9961,0.9953,0.9953
Mean,0.9976,1.0000,0.9869,1.0000,0.9934,0.9919,0.9919
Std,0.0013,0.0000,0.0072,0.0000,0.0036,0.0045,0.0044


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9764,0.9977,0.9614,0.9154,0.9379,0.9233,0.9237
1,0.9664,0.9977,0.9691,0.8655,0.9144,0.8936,0.8958
2,0.9721,0.9971,0.9614,0.8957,0.9274,0.9102,0.9110
3,0.9764,0.9979,0.9884,0.8951,0.9394,0.9249,0.9266
4,0.9757,0.9972,0.9538,0.9185,0.9358,0.9209,0.9211
Mean,0.9734,0.9975,0.9668,0.8981,0.9310,0.9146,0.9157
Std,0.0038,0.0003,0.0118,0.0189,0.0093,0.0117,0.0112


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9943,0.9875,0.9768,0.9922,0.9844,0.9809,0.9810
1,0.9921,0.9877,0.9807,0.9769,0.9788,0.9740,0.9740
2,0.9914,0.9843,0.9730,0.9805,0.9767,0.9715,0.9715
3,0.9971,0.9968,0.9961,0.9885,0.9923,0.9906,0.9906
4,0.9936,0.9886,0.9808,0.9846,0.9827,0.9787,0.9787
Mean,0.9937,0.9890,0.9815,0.9845,0.9830,0.9791,0.9791
Std,0.0020,0.0042,0.0079,0.0054,0.0054,0.0066,0.0066


...

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9821,0.9998,1.0000,0.9120,0.9540,0.9429,0.9445
1,0.9771,0.9997,1.0000,0.8900,0.9418,0.9277,0.9301
2,0.9786,0.9997,0.9961,0.8990,0.9451,0.9318,0.9337
3,0.9793,0.9998,1.0000,0.8993,0.9470,0.9342,0.9362
4,0.9829,0.9998,1.0000,0.9155,0.9559,0.9453,0.9467
Mean,0.9800,0.9998,0.9992,0.9032,0.9487,0.9364,0.9382
Std,0.0022,0.0001,0.0015,0.0093,0.0053,0.0067,0.0063


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9976,1.0000,0.9869,1.0000,0.9934,0.9919,0.9919,0.0260
svm,SVM - Linear Kernel,0.9967,0.0000,0.9915,0.9908,0.9911,0.9891,0.9891,0.0220
dt,Decision Tree Classifier,0.9937,0.9890,0.9815,0.9845,0.9830,0.9791,0.9791,0.0280
rf,Random Forest Classifier,0.9936,0.9998,0.9792,0.9861,0.9826,0.9786,0.9787,0.2700
lda,Linear Discriminant Analysis,0.9800,0.9998,0.9992,0.9032,0.9487,0.9364,0.9382,1.5560
nb,Naive Bayes,0.9734,0.9975,0.9668,0.8981,0.9310,0.9146,0.9157,0.0240


Durées d'entraînement par modèle : {'lr': 10.45185375213623, 'nb': 8.69553518295288, 'dt': 1.7269411087036133, 'rf': 3.0319671630859375, 'svm': 1.3875086307525635, 'lda': 1.5224056243896484}
Modèle sélectionné : LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9993,1.0000,1.0000,0.9962,0.9981,0.9976,0.9976
1,0.9979,1.0000,0.9923,0.9961,0.9942,0.9929,0.9929
2,0.9979,1.0000,0.9884,1.0000,0.9942,0.9929,0.9929
3,0.9993,1.0000,0.9961,1.0000,0.9981,0.9976,0.9976
4,0.9993,1.0000,1.0000,0.9962,0.9981,0.9976,0.9976
Mean,0.9987,1.0000,0.9954,0.9977,0.9965,0.9957,0.9957
Std,0.0007,0.0000,0.0045,0.0019,0.0019,0.0023,0.0023


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Modèle optimisé : LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
